In [1]:
import os
import pandas as pd
import warnings
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from catboost import CatBoostClassifier

%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']

In [2]:
path = '/home/kesci/data/competition_A/'
train_df = pd.read_csv(path+'train_set.csv') 
test_df  = pd.read_csv(path+'test_set.csv') 
submission  =  pd.read_csv(path+'submission_example.csv') 
print('Train Shape:{}\nTest Shape:{}'.format(train_df.shape,test_df.shape))
train_df.head()

Train Shape:(6000, 31)
Test Shape:(2785, 30)


,年龄,性别,区域,体重,身高,体重指数,肥胖腰围,腰围,最高血压,最低血压,好胆固醇,坏胆固醇,总胆固醇,血脂异常,PVD,体育活动,教育,未婚,收入,护理来源,视力不佳,饮酒,高血压,家庭高血压,糖尿病,家族糖尿病,肝炎,家族肝炎,慢性疲劳,ALF,ID
0,58,F,east,75.6,174.9,24.71,0.0,94.8,100.0,52.0,35.0,95.0,130.0,0,0,1.0,0.0,0.0,0.0,Private Hospital,0.0,0,1.0,1,0.0,1,1.0,0.0,1.0,0.0,4379
1,85,F,east,66.3,166.1,24.03,0.0,89.6,134.0,84.0,59.0,153.0,212.0,0,0,2.0,0.0,0.0,0.0,Private Hospital,0.0,1,0.0,0,0.0,0,0.0,0.0,0.0,NaN,7623
2,32,F,east,109.9,173.2,36.64,1.0,111.7,124.0,84.0,39.0,133.0,172.0,0,0,2.0,1.0,0.0,1.0,Private Hospital,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,1764
3,22,M,east,58.7,171.3,20.00,0.0,78.0,104.0,56.0,48.0,98.0,146.0,0,0,2.0,1.0,1.0,0.0,Never Counsulted,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,5450
4,44,F,west,79.7,172.1,26.91,0.0,93.8,114.0,60.0,34.0,195.0,229.0,0,0,2.0,0.0,0.0,0.0,Private Hospital,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,5196


In [3]:
train_df.describe()

,年龄,体重,身高,体重指数,肥胖腰围,腰围,最高血压,最低血压,好胆固醇,坏胆固醇,总胆固醇,血脂异常,PVD,体育活动,教育,未婚,收入,视力不佳,饮酒,高血压,家庭高血压,糖尿病,家族糖尿病,肝炎,家族肝炎,慢性疲劳,ALF,ID
count,6000.000000,5860.000000,5864.000000,5791.000000,5791.000000,5776.000000,5781.000000,5727.000000,5994.000000,5993.000000,5993.000000,6000.000000,6000.00000,5992.000000,5986.000000,5692.000000,5237.000000,5598.000000,6000.000000,5951.000000,6000.000000,6000.000000,6000.000000,5984.000000,5995.000000,5974.000000,4119.000000,6000.000000
mean,49.331000,79.140884,167.037142,28.306374,0.317044,96.837846,125.582944,71.469356,51.627794,152.999166,204.628066,0.105833,0.04150,2.017190,0.434347,0.366831,0.418178,0.065916,0.301000,0.401613,0.239500,0.110333,0.314500,0.067179,0.020684,0.031637,0.074775,4362.312833
std,18.784868,19.293747,10.124373,6.136871,0.465365,14.928330,21.004949,12.663285,15.518744,42.745425,42.707270,0.307650,0.19946,0.812113,0.495712,0.481982,0.493307,0.248158,0.458731,0.490266,0.426814,0.313331,0.464355,0.250353,0.142336,0.175047,0.263061,2538.515901
min,20.000000,33.700000,130.400000,14.420000,0.000000,58.600000,72.000000,10.000000,12.000000,27.000000,72.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33.000000,65.600000,159.700000,24.080000,0.000000,86.300000,111.000000,64.000000,41.000000,124.000000,176.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2142.750000
50%,47.000000,76.700000,166.600000,27.390000,0.000000,96.300000,122.000000,72.000000,49.000000,150.000000,201.000000,0.000000,0.00000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4363.500000
75%,65.000000,89.525000,174.200000,31.390000,1.000000,106.100000,136.000000,79.000000,60.000000,178.000000,230.000000,0.000000,0.00000,2.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,6558.250000
max,85.000000,191.100000,200.100000,66.440000,1.000000,166.000000,233.000000,132.000000,160.000000,684.000000,727.000000,1.000000,1.00000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8784.000000


In [4]:
train_df.columns

Index(['年龄', '性别', '区域', '体重', '身高', '体重指数', '肥胖腰围', '腰围', '最高血压', '最低血压',
       '好胆固醇', '坏胆固醇', '总胆固醇', '血脂异常', 'PVD', '体育活动', '教育', '未婚', '收入', '护理来源',
       '视力不佳', '饮酒', '高血压', '家庭高血压', '糖尿病', '家族糖尿病', '肝炎', '家族肝炎', '慢性疲劳',
       'ALF', 'ID'],
      dtype='object')

In [5]:
num_columns = ['年龄','体重','身高','体重指数', '腰围', '最高血压', '最低血压',
                '好胆固醇', '坏胆固醇', '总胆固醇','收入']
zero_to_one_columns = ['肥胖腰围','血脂异常','PVD']
str_columns = ['性别','区域','体育活动','教育','未婚','护理来源','视力不佳','饮酒','高血压',
                '家庭高血压', '糖尿病', '家族糖尿病','家族肝炎', '慢性疲劳','ALF']

In [6]:
g = sns.FacetGrid(train_df, col='肝炎')
g.map(plt.hist, '年龄', bins=20)

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


<Figure size 432x216 with 2 Axes>

In [7]:
# 空值填充
train_df.fillna(0,inplace=True)
test_df.fillna(0,inplace=True)

In [8]:
# 字符编码
for i in tqdm(str_columns):
    lbl = LabelEncoder()
    train_df[i] = lbl.fit_transform(train_df[i].astype(str))
    test_df[i]  = lbl.fit_transform(test_df[i].astype(str))

100%|██████████| 15/15 [00:00<00:00, 210.24it/s]


In [9]:
# 数值归一化
train_df[num_columns] = MinMaxScaler().fit_transform(train_df[num_columns])
test_df[num_columns]  = MinMaxScaler().fit_transform(test_df[num_columns])

In [10]:
all_columns = [i for i in train_df.columns if i not in ['肝炎','ID']]

train_x,train_y = train_df[all_columns].values,train_df['肝炎'].values
test_x  = test_df[all_columns].values
submission['hepatitis'] = 0

In [11]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
model = CatBoostClassifier(
    iterations=200,
    learning_rate=0.1,
    loss_function='Logloss'
    )
for train, valid in kfold.split(train_x, train_y):
    X_train, Y_train = train_x[train], train_y[train]
    X_valid, Y_valid = train_x[valid], train_y[valid]
    model.fit(X_train,Y_train, eval_set=(X_valid, Y_valid),use_best_model=True)
    Y_valid_pred_prob = model.predict_proba(X_valid)
    submission['hepatitis'] += model.predict_proba(test_x)[:,1] / 5

0:	learn: 0.5476648	test: 0.5450950	best: 0.5450950 (0)	total: 1.2s	remaining: 3m 59s
1:	learn: 0.4542235	test: 0.4506240	best: 0.4506240 (1)	total: 2.9s	remaining: 4m 47s
2:	learn: 0.3913477	test: 0.3892482	best: 0.3892482 (2)	total: 4.7s	remaining: 5m 8s
3:	learn: 0.3462174	test: 0.3450788	best: 0.3450788 (3)	total: 6.1s	remaining: 4m 58s
4:	learn: 0.3067070	test: 0.3065037	best: 0.3065037 (4)	total: 7.61s	remaining: 4m 56s
5:	learn: 0.2819000	test: 0.2817510	best: 0.2817510 (5)	total: 9.3s	remaining: 5m
6:	learn: 0.2628941	test: 0.2631047	best: 0.2631047 (6)	total: 11.1s	remaining: 5m 6s
7:	learn: 0.2496631	test: 0.2502320	best: 0.2502320 (7)	total: 12.8s	remaining: 5m 7s
8:	learn: 0.2325814	test: 0.2338527	best: 0.2338527 (8)	total: 14.3s	remaining: 5m 3s
9:	learn: 0.2206442	test: 0.2229252	best: 0.2229252 (9)	total: 15.7s	remaining: 4m 58s
10:	learn: 0.2115983	test: 0.2144001	best: 0.2144001 (10)	total: 17.2s	remaining: 4m 55s
11:	learn: 0.2074506	test: 0.2107228	best: 0.2107228 (

In [ ]:
!wget -nv -O kesci_submit https://cdn.kesci.com/submit_tool/v4/kesci_submit&&chmod +x kesci_submit
submission.to_csv('submission.csv',index=False)
!./kesci_submit -token '你的队伍Token' -file '/home/kesci/work/submission.csv'